# The Old Way
## Import Packages

In [19]:
import datetime as dt
import pandas as pd
import numpy as np
from pandas_datareader import data
import statsmodels.api as sm

## Load Universe
I created a CSV listing all stocks included in SPDR’s sector funds as of January 24th, 2016. It is loaded as a pandas dataframe. In order to make calls to Yahoo, all periods are replaced with hyphens.

We also pull all sector tickers and SPY as a proxy for the market as a whole.

In [2]:
universe = pd.read_csv('inputs/universe.csv', index_col='symbol')
universe.index = map(lambda x: x.replace('.', '-'), universe.index)

## Pull Price History

Price history is pulled from Yahoo and concatenated into a single multi-index dataframe. Stocks were pulled from January 1st 2010 through December 31st, 2015.

In [3]:
def pull_universe(uni, sdate, edate):
    '''Expects a universe csv. Returns a dataframe of prices.
    
    Might not work if an error is generated on the first stock in the index.
    Performance and stability could be improved through list comprehension
    and a single pd.concat operation.'''
    
    # Start by pulling one stock.
    df = data.DataReader(uni.index[0],data_source='yahoo', start=sdate, end=edate)
    df['symbol'] = uni.index[0]
    df['sector'] = uni.loc[uni.index[0],'sector']
    
    #Loop through remaining stocks. Append to prices file.
    for Tic in uni.index[1:]:
        try:
            df_new = data.DataReader(Tic,data_source='yahoo', start=sdate, end=edate)
            df_new['symbol'] = Tic
            df_new['sector'] = uni.loc[Tic,'sector']
            df = pd.concat([df, df_new])
        except IOError:
            pass

    return df

In [4]:
prices = pull_universe(uni=universe.iloc[:3, ], sdate=dt.datetime(2010, 1, 1), edate=dt.datetime(2015, 12, 31))

## Adjust and Calculate Returns

In [5]:
def adjust_prices(df):
    '''Adjusts df prices for dividends and splits.'''
    df['AdjFactor'] = df.groupby('symbol')['Adj Close'].shift(1) / df.groupby('symbol')['Close'].shift(1)
    df['Open'] = df['Open'] * df['AdjFactor']
    df['High'] = df['High'] * df['AdjFactor']
    df['Low'] = df['Low'] * df['AdjFactor']
    df['Close'] = df['Close'] * df['AdjFactor']
    new_df = df[['sector', 'symbol', 'Open', 'High', 'Low', 'Close', 'Volume']]
    new_df.columns = ['sector', 'symbol', 'open', 'high', 'low', 'close', 'volume']
    return new_df

def add_returns(df):
    df['lastClose'] = df.groupby('symbol')['close'].shift(1)
    df['ret_cc'] = np.log(df['close'] / df['lastClose'])
    df['ret_oc'] = np.log(df['close'] / df['open'])
    df['ret_co'] = np.log(df['open'] / df['lastClose'])
    df.drop('lastClose', 1, inplace=True)
    return df

prices = adjust_prices(prices)
prices = add_returns(prices)

## Beta Lags
In its simplest form, the capital asset pricing model (CAPM) predicts that a security's expected return over a given period of time is equal to the return of the market times a beta parameter (usually set by linear regression analysis). Modifications of this model include:

1. Allowing for "excess return" (alpha)
1. Utilizing "excess return," i.e., subtracting the risk free rate from all returns.
1. Adding additional factors for the return of a stock's industry.
1. Considering financial indicators of the health of the target company (price book ratios, market cap size, etc).

When valuing small companies, many professional valuators believe that a "beta lag" exists. That is, they believe that last period's market return impacts this perod's security return. This effect is claimed to be more prevalent in smaller, less liquid securities.

If this is true, it should be trivial to develop strategies which take advantage of this effect.

In [21]:
prices.head()

,sector,symbol,open,high,low,close,volume,ret_cc,ret_oc,ret_co
Date,,,,,,,,,,
2010-01-04,XLP,PG,NaN,NaN,NaN,NaN,9190800,NaN,NaN,NaN
2010-01-05,XLP,PG,49.741287,49.863339,49.310025,49.749422,8649400,NaN,0.000164,NaN
2010-01-06,XLP,PG,49.586683,49.602957,49.228657,49.513449,9908400,-0.004755,-0.001478,-0.003277
2010-01-07,XLP,PG,49.293754,49.391397,49.017097,49.244931,8972800,-0.005438,-0.000991,-0.004447
2010-01-08,XLP,PG,49.114740,49.253067,48.870631,49.179834,8464600,-0.001323,0.001324,-0.002647


In [18]:
def simple_beta(stocks, history):
    to_analyze = list(set(history['symbol'])) # Convert to set to remove duplicates.
    stocks['alpha'] = np.nan
    stocks['beta'] = np.nan
    
    for ticker in to_analyze:
        temp_model = sm.OLS
        stocks.loc[ticker, 'alpha'] = 0
        stocks.loc[ticker, 'beta'] = 0
    
    return universe

universe = simple_beta(universe, prices)

universe.head(10)

,company_name,index_weight,sector,sector_name,alpha,beta
PG,Procter & Gamble,11.90%,XLP,Consumper Staples,0.0,0.0
KO,Coca-Cola Co,9.31%,XLP,Consumper Staples,0.0,0.0
PM,Philip Morris International,7.61%,XLP,Consumper Staples,0.0,0.0
CVS,CVS Health Corporation,6.16%,XLP,Consumper Staples,NaN,NaN
MO,Altria Group Inc,5.84%,XLP,Consumper Staples,NaN,NaN
WMT,Wal-Mart Stores,5.56%,XLP,Consumper Staples,NaN,NaN
PEP,PepsiCo Inc,4.53%,XLP,Consumper Staples,NaN,NaN
WBA,Walgreens Boots Alliance Inc,4.24%,XLP,Consumper Staples,NaN,NaN
COST,Costco Wholesale Corp,3.96%,XLP,Consumper Staples,NaN,NaN
MDLZ,Mondelez International Inc,3.53%,XLP,Consumper Staples,NaN,NaN


# Let's do that all again with the package
## Import Packages

In [6]:
import wmcm.functions as wmf

In [7]:
wmf.greg()

Fuck off!
